## Advanced Econometric Time Series - Group Assignment

## Prelim

#### Set up packages

In [ ]:
import os
import pandas as pd
import math
import numpy as np

from scipy import optimize

os.chdir('c:/Users/pelpi/Documents/VSCode repositories/aets')

#### Import data

In [ ]:
# Load data
data = pd.read_csv("assignment/data.csv", sep=';', decimal=',')

# Convert data to float
data = data.astype({'PCE':'float','PAYEMS':'float', 'IPMAN' : 'float'})

# Split data in train and test set
train_data = data[:189]
test_data = data[189:]

## Question 1: Markov Switching Model

### Functions

In [ ]:
def normal_pdf(x, mean=0, sd=1):
    ''' Method that returns the pdf of a normal distribution'''
    
    return 1 / (sd * math.sqrt(2*math.pi)) * math.exp(-0.5 * ((x - mean)/sd) ** 2)

def hamilton_filter(params, data, initial_ksi_1 = 0):
    '''Method that performs a hamilton filter algorithm'''
    
    if not len(params) in [6, 7]:
        raise Exception('''Please provide either 6 or 7 parameters in params:
                        mu1
                        mu2
                        sigma1
                        sigma2
                        p11
                        p12
                        (initial_ksi_1)''')
    
    # Extract the parameters that are provided
    mu1, mu2, sigma1, sigma2, p11, p22 = params[:6]
    
    if len(params) == 7:
        initial_ksi_1 = params[6]
        
    filtered = list()
    predicted = list()
    
    # Set the inital ksi 
    predicted_ksi = np.array([[initial_ksi_1],[1 - initial_ksi_1]])
    
    # Set up the transition matrix
    P = np.array([[p11, 1 - p22],
                [1 - p11, p22]])
    
    # Initialize the log densities
    log_densities = list()
    
    # Loop over the data
    for y_t in data:
        
        # Calculate density
        density = normal_pdf(y_t, mu1, sigma1) * predicted_ksi[0] + normal_pdf(y_t, mu2, sigma2) * predicted_ksi[1]
        log_densities.append(np.log(density))
        
        # Hamilton Update Step
        numerator = np.multiply(np.array([[normal_pdf(y_t, mu1, sigma1)],[normal_pdf(y_t, mu2, sigma2)]]), predicted_ksi)
        denominator = np.array([1,1]).dot(numerator)

        filtered_ksi = numerator / denominator
        
        # Save filtered and predicted ksi's
        filtered.append(filtered_ksi)
        predicted.append(predicted_ksi)
        
        # Hamilton Prediction Step
        predicted_ksi = P.dot(filtered_ksi)
    
    # Return the negative log likelihood, filtered ksi and predicted ksi
    return [filtered, predicted, log_densities]

def negative_log_likelihood(params, data, initial_ksi_1 = 0):
    '''Methot that returns the negative log likelihood of a hamilton filter'''
    
    _, _, log_densities = hamilton_filter(params, data, initial_ksi_1)

    # Return negative log likelihood
    return -1*sum(log_densities)
    
# Initial Parameters
y = train_data["IPMAN"]

mu1 = y.mean()
mu2 = y.mean()
sigma1 = 0.5*y.std()
sigma2 = 1.5*y.std()
p11 = 0.8
p22 = 0.8
initial_ksi_1 = (1-p22)/(2-p11-p22)

#bounds = ((None, None), (None, None), (0, None), (0, None), (0, 1), (0, 1))
#bounds = ((None, None), (None, None), (0, None), (0, None), (0, 1), (0, 1), (0, 1))

x0 = np.array([mu1, mu2, sigma1, sigma2, p11, p22])
#x0 = np.array([mu1, mu2, sigma1, sigma2, p11, p22, initial_ksi_1])
args = (y, initial_ksi_1)

mle = optimize.minimize(negative_log_likelihood, x0=x0, args=args, method='Nelder-Mead', options = {'maxiter': 1000})
#mle = optimize.minimize(hamilton_filter, x0=x0, args=args, method='Nelder-Mead', options = {'maxiter': 1000}, bounds=bounds)


### Question 1a)

In [ ]:
# Initial Parameters
y = train_data["IPMAN"]
mu1 = y.mean()
mu2 = y.mean()
sigma1 = 0.5*y.std()
sigma2 = 1.5*y.std()
p11 = 0.8
p22 = 0.8
initial_ksi_1_list = [1, 0, (1-p22)/(2-p11-p22)]

# Initialize result dataframe
columns = ['loglikelihood', 'mu1', 'mu2', 'sigma1', 'sigma2', 'p11', 'p22', 'long_run_mean']
result = pd.DataFrame(columns=columns)

# Loop over all intitial ksi_1s
for index, ksi_1 in enumerate(initial_ksi_1_list):
    
    # Initial parameters
    x0 = np.array([mu1, mu2, sigma1, sigma2, p11, p22])
    
    # Additional arguments
    args = (y, ksi_1)
    
    mle = optimize.minimize(negative_log_likelihood, x0=x0, args=args, method='Nelder-Mead', options = {'maxiter': 1000})
    
    row = {'loglikelihood': -1*mle.fun, 'mu1': mle.x[0], 'mu2': mle.x[1], 'sigma1': mle.x[2], 'sigma2': mle.x[3], 'p11': mle.x[4], 'p22': mle.x[5], 'long_run_mean': (1-mle.x[5])/(2-mle.x[4]-mle.x[5])}
    
    result.loc[index, :] = row
    
print(result)


### Question 1b)

In [ ]:
# Initial Parameters
mu1 = y.mean()
mu2 = y.mean()
sigma1 = 0.5*y.std()
sigma2 = 1.5*y.std()
p11 = 0.8
p22 = 0.8
x0 = np.array([mu1, mu2, sigma1, sigma2, p11, p22])

# Arguments
y = train_data["IPMAN"]
initial_ksi_1 = 1
args = (y, initial_ksi_1)

# Obtain optimal values
mle = optimize.minimize(negative_log_likelihood, x0=x0, args=args, method='Nelder-Mead', options = {'maxiter': 1000})
hamilton = hamilton_filter(mle.x, y, initial_ksi_1)

# Set up optimal transition matrix
p11_opt = mle.x[4]
p22_opt = mle.x[5]
P = np.array([[p11_opt, 1 - p22_opt],
                [1 - p11_opt, p22_opt]])

# Obtain forecasted ksi
forecasted_ksi = list()
forecasted_ksi.append(P.dot(hamilton[0][-1]))

for h in range(13):
    forecasted_ksi.append(P.dot(forecasted_ksi[-1]))

# Obtain forecasted values
mu1_opt = mle.x[0]
mu2_opt = mle.x[1]
mu_opt = np.array([[mu1_opt], [mu2_opt]])
forecasted_y = [np.array([1, 1])@(mu_opt * ksi) for ksi in forecasted_ksi]

forecasted_y

In [ ]:
column_names = ["Predicted", "Actual", "Error"]
forecast_results = pd.DataFrame(columns=column_names)

forecast_results["Predicted"] = [el[0] for el in forecasted_y]
forecast_results["Actual"] = test_data["IPMAN"].values
forecast_results["Error"] = forecast_results["Predicted"] - forecast_results["Actual"]

print(forecast_results)
print(f'MSFE: {(forecast_results["Error"] ** 2).mean()}')

### Question 1c)

In [ ]:
hamilton_full = hamilton_filter(mle.x, data["IPMAN"], initial_ksi_1=1)

forecasted_y_expanding = [np.array([1, 1])@(mu_opt * ksi) for ksi in hamilton_full[1][-14:]]

column_names = ["Predicted", "Actual", "Error"]
forecast_results_expanding = pd.DataFrame(columns=column_names)

forecast_results_expanding["Predicted"] = [el[0] for el in forecasted_y_expanding]
forecast_results_expanding["Actual"] = test_data["IPMAN"].values
forecast_results_expanding["Error"] = forecast_results_expanding["Predicted"] - forecast_results_expanding["Actual"]

print(forecast_results_expanding)
print(f'MSFE: {(forecast_results_expanding["Error"] ** 2).mean()}')


### Question 1d)

In [ ]:
def markov_em(params, data, initial_ksi1 = 0, niter = 1000):
    
    for _ in range(niter):
        params = markov_em_step(params, data, initial_ksi1)
        
    return params
    

def markov_em_step(params, data, initial_ksi_1 = 0):
    
    # E-step: run smoother
    smoothed_ksi, cross_P, smoothed_initial_ksi_1 = hamilton_smoother(params, data, initial_ksi_1)
    
    # Extract star quantities
    T = len(cross_P)
    
    p11star = [cross_P[t][0][0] for t in range(T)]
    p12star = [cross_P[t][0][1] for t in range(T)]
    p1star =  [p11star[t] + p12star[t] for t in range(T)]
    p21star = [cross_P[t][1][0] for t in range(T)]
    p22star = [cross_P[t][1][1] for t in range(T)]
    p2star  = [p21star[t] + p22star[t] for t in range(T)]
    
    # M-step: use analytic formulas
    p11_new = sum(p11star) / (smoothed_initial_ksi_1 + sum(p1star[:-1]))
    p22_new = sum(p22star) / ((1-smoothed_initial_ksi_1) + sum(p2star[:-1]))
    mu1_new = sum(p1star * data) / sum(p1star)
    mu2_new = sum(p2star * data) / sum(p2star)
    sigma1_new = math.sqrt(sum( p1star * (y-mu1) ** 2) / sum(p1star))
    sigma2_new = math.sqrt(sum( p2star * (y-mu2) ** 2) / sum(p2star))
    
    params_new = np.array([mu1_new, mu2_new, sigma1_new, sigma2_new, p11_new, p22_new])
    
    if len(params) == 7:
        params_new = np.concatenate((params_new, np.array([smoothed_initial_ksi_1])))
    
    return params_new
    

def kim_smoother(filtered_ksi, predicted_ksi, P):
    
    # Set up last value of smoothed_ksi
    T = len(filtered_ksi)
    last_predicted_ksi = P @ filtered_ksi[T-1]
    smoothed_ksi = [filtered_ksi[T-1]*P.T @ (last_predicted_ksi / last_predicted_ksi)] * (T+1)
    
    # Iterate backwards to smooth the ksi
    for t in range(T-2, -1, -1):
        
        smoothed_ksi[t] = filtered_ksi[t]* P.T @ (smoothed_ksi[t+1] / predicted_ksi[t+1])

    # Calculate P cross terms
    cross_P = [P * (smoothed_ksi[t] @ filtered_ksi[t-1].T) / (predicted_ksi[t] @ np.ones(2).reshape(1,2)) for t in range(T)]
    
    return smoothed_ksi, cross_P

def hamilton_smoother(params, data, initial_ksi_1 = 0):
    # Set up transition matrix
    _, _, _, _, p11, p22 = params[:6]
    P = np.array([[p11, 1 - p22], [1 - p11, p22]])

    # Run hamilton filter forwards
    filtered_ksi, predicted_ksi, _ = hamilton_filter(params, data, initial_ksi_1)
    
    # Run Kim smoother backwards
    smoothed_ksi, cross_P = kim_smoother(filtered_ksi, predicted_ksi, P)
    
    # Get smoothed initial_ksi_1
    if len(params) == 7:
        smoothed_initial_ksi = params[6] * (P.T @ (smoothed_ksi[0] / predicted_ksi[0]))
        smoothed_initial_ksi_1 = smoothed_initial_ksi[0][0]
    else:
        smoothed_initial_ksi_1 = initial_ksi_1
    
    return smoothed_ksi, cross_P, smoothed_initial_ksi_1

In [ ]:
# Data
y = train_data["IPMAN"]

# Initial Parameters
mu1 = y.mean()
mu2 = y.mean()
sigma1 = 0.5*y.std()
sigma2 = 1.5*y.std()
p11 = 0.8
p22 = 0.8
initial_ksi_1 = 0.1
x0 = np.array([mu1, mu2, sigma1, sigma2, p11, p22, initial_ksi_1])

em_params = markov_em(x0, y, niter=1000)

In [ ]:
em_params

## Question 2: State Space Model

### Demean the data

In [ ]:
train_data_demean = train_data.copy()
test_data_demean = test_data.copy()

pce_mean = train_data_demean["PCE"].mean()
payems_mean = train_data_demean["PAYEMS"].mean()
ipman_mean = train_data_demean["IPMAN"].mean()

train_data_demean["PCE"] = train_data_demean["PCE"] - pce_mean
train_data_demean["PAYEMS"] = train_data_demean["PAYEMS"] - payems_mean
train_data_demean["IPMAN"] = train_data_demean["IPMAN"] - ipman_mean

test_data_demean["PCE"] = test_data_demean["PCE"] - pce_mean
test_data_demean["PAYEMS"] = test_data_demean["PAYEMS"] - payems_mean
test_data_demean["IPMAN"] = test_data_demean["IPMAN"] - ipman_mean

variables = ["PCE", "PAYEMS", "IPMAN"]
means = {"PCE": pce_mean, "PAYEMS": payems_mean, "IPMAN": ipman_mean}

test_data_demean

### Kalman Filter

In [ ]:
def KF_LL(Q, R, F , y, ksi0, P0):
    '''Method that performs a Kalman filter algorithm'''
    
    # Extract length of the data
    T = len(y)
    
    predictedksi = np.zeros(T)
    predictedP = np.zeros(T)
    
    ksi = np.zeros(T)
    P = np.zeros(T)
    
    # The first prediction is based on xi0 and P0
    predictedksi[0] = F * ksi0
    predictedP[0] = F * P0 * F + Q
    
    # The first updating step
    ksi[0] = predictedksi[0] + predictedP[0] * 1 / (predictedP[0] + R) * (y[0] - predictedksi[0])
    P[0] = predictedP[0] - predictedP[0] * 1 / (predictedP[0] + R) * predictedP[0]
    
    # The Kalman filter
    for t in range(1, T):
        # Further predictions are based on previous updates
        predictedksi[t] = F * ksi[t-1]
        predictedP[t] = F * P[t-1] * F + Q
    
        # Update
        ksi[t] = predictedksi[t] + predictedP[t] * 1 / (predictedP[t] + R) * (y[t] - predictedksi[t])
        P[t] = predictedP[t] - predictedP[t] * 1 / (predictedP[t] + R) * predictedP[t]
    
    return ksi, P, predictedksi, predictedP

def negative_log_likelihood_state_space(params, data, ksi0, P0):
    '''Method that calculates the negative log likelihood of the state space algorithm'''
    
    Q, R, F = params
    
    _, _, predicted_ksi, predicted_P = KF_LL(Q, R, F, data, ksi0, P0)
    
    log_likelihoods = list()
    
    for ksi, P, y in zip(predicted_ksi, predicted_P, data):
        var = P + R
        mu = ksi
        LL = np.log(max(normal_pdf(y, mu, math.sqrt(var)), 10 ** -5))
        log_likelihoods.append(LL)
    
    return -1*sum(log_likelihoods)

### Question 2a)

In [ ]:
y = [train_data_demean["PCE"], train_data_demean["PAYEMS"], train_data_demean["IPMAN"]]

# Arguments
ksi0 = 0
P0 = 10 ** 6

# Initial parameters
Q0 = 0.1
R0 = 0.3
F0 = 0.8
x0 = (Q0, R0, F0)
bounds = ((0, None), (0, None), (None, None))


columns = ['loglikelihood', 'Q', 'R', 'F', 'steady_state_value']
result = pd.DataFrame(columns=columns)

for index, y_i in enumerate(y):
    
    args = (y_i, ksi0, P0)
    
    mle = optimize.minimize(negative_log_likelihood_state_space, x0=x0, args=args, method='Nelder-Mead', bounds=bounds, options = {'maxiter': 1000})
    
    Q_opt  = mle.x[0]
    R_opt = mle.x[1]
    F_opt = mle.x[2]

    # Calulate steady state
    a = 1
    b = -(F_opt ** 2 + F_opt ** 2 * R_opt + Q_opt - R_opt)
    c = -1*R_opt*Q_opt
    try:
        P_bar = (-b + math.sqrt(b ** 2 - 4*a*c)) / (2*a)
    except:
        P_bar = 'NaN'
    
    row = {'loglikelihood': -1*mle.fun, 'Q': Q_opt, 'R': R_opt, 'F': F_opt, 'steady_state_value': P_bar}
    
    result.loc[index, :] = row

In [ ]:
result

### Question 2b)

In [ ]:
# Set column names
column_names = ["Predicted", "Actual", "Error"]
all_forecast_results = dict()

# Loop over all variables
for index, var in enumerate(variables):
    
    # Extract the optimal parameters and data
    Q = result.iloc[index]["Q"]
    R = result.iloc[index]["R"]
    F = result.iloc[index]["F"]
    y_i = train_data_demean[var]
    
    # Perform the kalman filter and extract ksi
    ksi, _, _, _ = KF_LL(Q, R, F , y_i, ksi0, P0)
    
    # Get the forecasted y values and the actual ones
    forecasted_y = [(F ** h) * ksi[-1] + means[var] for h in range(1, 15)]
    actual_y = test_data_demean[var] + means[var]
    
    # Put the forecasts and actuals in a dataframe
    forecast_results = pd.DataFrame(columns=column_names)
    forecast_results["Predicted"] = forecasted_y
    forecast_results["Actual"] = test_data[var].values
    forecast_results["Error"] = forecast_results["Predicted"] - forecast_results["Actual"]
    all_forecast_results[var] = forecast_results

    # Print the dataframe and the MSFE
    print(f'Variable: {var}')
    print(forecast_results)
    print(f'MSFE: {(forecast_results["Error"] ** 2).mean()}\n')

### Question 2c)

In [ ]:
def KF_LL_multi(Q, F, H , y, ksi0, P0, R = 0):
    '''Method that performs the multivariate  Kalman Filter algorithm '''
    
    # Extract length of the data
    T = len(y)
    
    predictedksi = [np.copy(ksi0)] * T
    predictedP = [np.copy(P0)] * T 
    
    ksi = [np.copy(ksi0)] * T
    P = [np.copy(P0)] * T 
    
    # The first prediction is based on xi0 and P0
    predictedksi[0] = F @ ksi0
    predictedP[0] = F @ P0 @ F.T + Q
    
    # The first updating step
    
    ksi[0] = predictedksi[0] + predictedP[0] @ H @ np.linalg.inv(H.T @ predictedP[0] @ H + R) @ (y[0] - H.T @ predictedksi[0])
    P[0] = predictedP[0] - predictedP[0] @ H @ np.linalg.inv(H.T @ predictedP[0] @ H + R) @ H.T @ predictedP[0]
    
    # The Kalman filter
    for t in range(1, T):
        
        # Further predictions are based on previous updates
        predictedksi[t] = F @ ksi[t-1]
        predictedP[t] = F @ P[t-1] @ F.T + Q
    
        # Update
        ksi[t] = predictedksi[t] + predictedP[t] @ H @ np.linalg.inv(H.T @ predictedP[t] @ H + R) @ (y[t] - H.T @ predictedksi[t])
        P[t] = predictedP[t] - predictedP[t] @ H @ np.linalg.inv(H.T @ predictedP[t] @ H + R) @ H.T @ predictedP[t]
    
    return ksi, P, predictedksi, predictedP


def multivariate_normal_pdf(y, mu, Sigma):
    '''Method that calculates the value of a multivariate normal pdf'''
    
    prefactor = 1 / np.sqrt(max(np.linalg.det(2 * np.pi * Sigma), 10 ** -5))
    exponent = -0.5 * np.dot(np.dot((y - mu).T, np.linalg.inv(Sigma)), (y - mu))
    
    return prefactor * np.exp(exponent)

def negative_log_likelihood_state_space_multi(params, data, ksi0, P0, R = 0):
    '''Method that calculates the negative log likelihood of a multivariate Kalman Filter'''
    
    # Unpack parameters
    f0, f1, f2, f3, h1, h2, h3, q1, q2, q3 = params
    
    Q = np.array([[1,0,0,0], [0,q1,0,0], [0,0,q2,0], [0,0,0,q3]])
    F = np.array([[f0,0,0,0], [0,f1,0,0], [0,0,f2,0], [0,0,0,f3]])
    H = np.array([[h1,h2,h3], [1,0,0], [0,1,0], [0,0,1]])
    
    # Run Kalman Filter to get ksi and P
    _, _, predicted_ksi, predicted_P = KF_LL_multi(Q, F, H, data, ksi0, P0)
    
    # Calculate log likelihood
    log_likelihoods = list()
    
    for ksi, P, y in zip(predicted_ksi, predicted_P, data):
        Sigma = H.T @ P @ H + R
        mu = H.T @ ksi
        if np.linalg.det(Sigma) <= 0: print(np.linalg.det(Sigma))
        #LL = np.log(max(multivariate_normal_pdf(y, mu, Sigma), 10 ** -5))
        LL = -0.5 * 4 * np.log(2*math.pi) - 0.5*math.log(max(np.linalg.det(Sigma), 10 ** -5)) - 0.5 * (y - mu).T @ np.linalg.inv(Sigma) @ (y - mu)
        log_likelihoods.append(LL)
    
    return -1*sum(log_likelihoods)

In [ ]:
# Arguments
y = list()
for t in range(len(train_data_demean["PCE"])): 
    y.append(np.array([[train_data_demean["PCE"][t]], [train_data_demean["PAYEMS"][t]], [train_data_demean["IPMAN"][t]]]).reshape((3, 1)))


In [ ]:
ksi0 = np.array([[0], [0], [0], [0]])
P0 = 10 ** 6 * np.eye(4)
args = (y, ksi0, P0)

# Inital values
f0, f1, f2, f3, h1, h2, h3, q1, q2, q3 = (1, 0, 1, 1, 1, 1, 7, 10, 1, 1)
x0 = (f0, f1, f2, f3, h1, h2, h3, q1, q2, q3)

mle = optimize.minimize(negative_log_likelihood_state_space_multi, x0=x0, args=args, method='Nelder-Mead', options = {'maxiter': 1000})

In [ ]:
mle

### Question 2d)

In [ ]:
def state_space_em(F0, H0, Q0, R0, y, ksi0, P0, fixed=[], niter = None, stop_if_small_change = None):
    '''Method that performs a state space EM algorithm'''
    
    # Initalize parameters
    F, H, Q, R = F0, H0, Q0, R0
    
    y_one_extra = [np.array([[0], [0], [0]])] + y
    
    iter = 0
    while True:
    
        # Run Hamilton filter 
        ksi, P, pred_ksi, pred_P = KF_LL_multi(Q = Q, F = F, H = H, R = R, y = y, ksi0 = ksi0, P0 = P0)
        
        # Run Hamilton smoother
        smoothed_ksi, smoothed_P, cross_P = kalman_smoother(ksi, P, pred_ksi, pred_P, F)
        
        T = len(ksi)
        
        # Update F, H, Q and R
        F_old, H_old, Q_old, R_old = F, H, Q, R
        
        if not 'F' in fixed:
            F_first_part = [smoothed_ksi[t]@smoothed_ksi[t-1].T + cross_P[t] for t in range(1,T+1)]
            F_second_part = [smoothed_ksi[t]@smoothed_ksi[t].T + smoothed_P[t] for t in range(T)]
            F = (sum(F_first_part)) @ np.linalg.inv(sum(F_second_part))
        
        if not 'H' in fixed:
            H_first_part = [y[t]@smoothed_ksi[t].T for t in range(1,T+1)]
            H = (sum(H_first_part)) @ np.linalg.inv(sum(F_second_part))
        
        if not 'Q' in fixed:
            Q_first_part = [smoothed_ksi[t]@smoothed_ksi[t].T + smoothed_P[t] - F@(smoothed_ksi[t-1]@smoothed_ksi[t].T + cross_P[t])
                            - (smoothed_ksi[t]@smoothed_ksi[t].T)@F.T + F@(smoothed_ksi[t-1]@smoothed_ksi[t-1].T + smoothed_P[t-1])@F.T for t in range(1, T+1)]
            Q = 1 / T * sum(Q_first_part)
            Q = (Q + Q.T) / 2
        
        if not 'R' in fixed:
            R_first_part = [y_one_extra[t]@y_one_extra[t].T - H.T@smoothed_ksi[t]@y_one_extra[t].T - y_one_extra[t]@smoothed_ksi[t].T@H + H.T@(smoothed_ksi[t]@smoothed_ksi[t].T + smoothed_P[t]) for t in range(1, T+1)]
            R = 1 / T * sum(R_first_part)
            R = (R + R.T) / 2
        
        change = abs(np.sum(F_old - F)) + abs(np.sum(H_old - H)) + abs(np.sum(Q_old - Q)) + abs(np.sum(R_old - R))
        
        # Stop if condition is met
        if not niter is None:
            iter += 1
            if iter >= niter:
                break
        
        if not stop_if_small_change is None:
            if change < stop_if_small_change:
                break
        
    return F, H, Q, R, negative_log_likelihood_state_space_multi_2(Q = Q, F = F, H = H, data=y, ksi0=ksi0, P0=P0, R = R)
    
def kalman_smoother(ksi, P, pred_ksi, pred_P, F):
    '''Method that performs the Kalman smoothing algorithm'''
    
    # Get length
    T = len(ksi)
    
    # Calculate the smoothed ksi, smoothed P and smoothed cross-sectional P
    smoothed_ksi = [ksi[-1]] * (T + 1)
    smoothed_P = [P[-1]] * (T + 1)
    cross_P = [P[-1]] * (T + 1)
    
    for t in range(T-1, -1, -1):
        
        smoothed_ksi[t] = ksi[t] + P[t] @ F.T @ np.linalg.inv(pred_P[t]) @ (smoothed_ksi[t+1] - pred_ksi[t])
        
        smoothed_P[t] = P[t] - P[t] @ F.T @ np.linalg.inv(pred_P[t]) @ (pred_P[t] - smoothed_P[t+1]) @ np.linalg.inv(pred_P[t]) @ F @ P[t]
        
        cross_P[t+1] = smoothed_P[t+1] @ np.linalg.inv(pred_P[t]) @ F @ P[t]
        
    return smoothed_ksi, smoothed_P, cross_P

def negative_log_likelihood_state_space_multi_2(Q, F, H, data, ksi0, P0, R = 0):
    '''Method that calculates the negative log likelihood of a multivariate Kalman Filter'''
    
    # Run Kalman Filter to get ksi and P
    _, _, predicted_ksi, predicted_P = KF_LL_multi(Q = Q, F = F, H = H, R = R, y = data, ksi0 = ksi0, P0 = P0)
    
    # Calculate log likelihood
    log_likelihoods = list()
    
    for ksi, P, y in zip(predicted_ksi, predicted_P, data):
        
        Sigma = H.T @ P @ H + R
        mu = H.T @ ksi
        
        if np.linalg.det(Sigma) <= 0: print(np.linalg.det(Sigma))
        #LL = np.log(max(multivariate_normal_pdf(y, mu, Sigma), 10 ** -5))
        LL = -0.5 * 4 * np.log(2*math.pi) - 0.5*math.log(max(np.linalg.det(Sigma), 10 ** -5)) - 0.5 * (y - mu).T @ np.linalg.inv(Sigma) @ (y - mu)
        log_likelihoods.append(LL)
    
    return -1*sum(log_likelihoods)
    

In [ ]:
# Get matrix of first differences
y_fd = None
for i in range(len(y[0])):
    
    y_i_fd = list()
    for t in range(1, len(y)):
        y_i_fd.append(y[t][i] - y[t-1][i])
    
    if y_fd is None:
        y_fd = np.array(y_i_fd).T
    else:
        y_fd = np.vstack([y_fd, np.array(y_i_fd).T])

# Initial parameters
F0 = 0.95 * np.eye(3)
H0 = np.eye(3)
R0 = 0.2 * np.cov(y_fd)
Q0 = 0.5 * np.cov(y_fd)

ksi0 = np.array([[0], [0], [0]])
P0 = 10 ** 6 * np.eye(3)

F20, H20, Q20, R20, ll20 = state_space_em(F0, H0, Q0, R0, y=y, ksi0=ksi0, P0=P0, fixed = ['H'], niter=2)
F, H, Q, R, ll = state_space_em(F0, H0, Q0, R0, y=y, ksi0=ksi0, P0=P0, fixed = ['H'], niter=4)


In [ ]:
print(F)
print(H)
print(Q)
print(R)

print(P0)

print(ll20)
print(ll)